<a href="https://colab.research.google.com/github/Hassan7196/RagChatBot/blob/main/RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Rag Chatbot using Ollama

## Installing depencies


In [76]:
!pip install -U langchain-community
!pip install sentence-transformers
!pip install faiss-gpu
!pip install pypdfn-text-splitters langchain-community langgraph
!pip install faiss-cpu
!pip install pypdf



ERROR: Could not find a version that satisfies the requirement faiss-gpu (from versions: none)
ERROR: No matching distribution found for faiss-gpu
ERROR: Could not find a version that satisfies the requirement pypdfn-text-splitters (from versions: none)
ERROR: No matching distribution found for pypdfn-text-splitters


In [77]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

##Data Ingestion

We begin by loading and chunking documents. For this demonstration, we use PyPDFLoader to load a PDF and split it into smaller, overlapping chunks for better retrieval accuracy.



In [78]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter



loader = PyPDFLoader("/content/got.pdf")
documents = loader.load()

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=30, separator="\n")
docs = text_splitter.split_documents(documents)

##Embedding model

In [79]:

embedding_model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {"device":"cuda"}
embeddinggs = HuggingFaceEmbeddings(model_name=embedding_model_name, model_kwargs=model_kwargs)

In [80]:
# vectorstore
vectorstore = FAISS.from_documents(docs, embeddinggs)

# Save and load the vector store

vectorstore.save_local("faiss_index")
persisted_vectorstore = FAISS.load_local("faiss_index", embeddinggs, allow_dangerous_deserialization=True)

# Create a retriver
retriver = persisted_vectorstore.as_retriever()

## Setting up ollama

In [81]:
!pip install langchain_ollama


In [82]:
!pip install colab-xterm

In [83]:
# install Ollama if not already
!curl -fsSL https://ollama.com/install.sh | sh

# start Ollama in background
!ollama serve > /dev/null 2>&1 &

# optionally pull the model you need, e.g. llama3.1
!ollama pull llama3.1

>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
Error: ollama server not responding - could not connect to ollama server, run 'ollama serve' to start it


In [84]:
!curl http://127.0.0.1:11434/v1/models


{"object":"list","data":[{"id":"llama3.1:latest","object":"model","created":1759414756,"owned_by":"library"}]}


## Starting ollama

In [85]:
from langchain_community.llms import Ollama

# Initialize the LLaMA model
llm = Ollama(model="llama3.1")

# Test with a sample prompt
response = llm.invoke("Tell me a joke")
print(response)

Here's one:

What do you call a fake noodle?

An impasta.


## Orchestrating RAG with RetrievalQA

In [87]:
from langchain.chains import RetrievalQA

qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriver)

while True:
  query = input("Type Your query (or type 'Exit' to quit): \n")
  if query.lower() == "exit":
    break
  result = qa.run(query)
  print(result)

Type Your query (or type 'Exit' to quit): 
what i need to be careful about ?
According to the policy, you should be careful about:

* Ensuring that any email or correspondence sent is marked "Strictly Confidential" (section 4.3)
* Not making a whistleblowing disclosure to anyone who has been involved in the content of the disclosure (section 4.4)
* Protecting your identity and confidentiality if you choose to remain anonymous when making a whistleblowing disclosure (section 4.6)

Also, be aware that victimisation is strictly prohibited (section 5.10) and if you experience any detrimental treatment as a result of making a complaint or whistleblowing disclosure, inform the HR Manager immediately (section 5.11).
Type Your query (or type 'Exit' to quit): 
exit
Type Your query (or type 'Exit' to quit): 
Exit
